# Generate a dataset for preference alignment

This notebook will guide you through the process of generating a dataset for preference alignment. We'll use the `distilabel` package to generate a dataset for preference alignment.

So let's dig in to some preference alignment datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Generate a dataset for preference alignment</h2>
    <p>Now that you've seen how to generate a dataset for preference alignment, try generating a dataset for preference alignment.</p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Generate a dataset for preference alignment</p>
    <p>🐕 Generate a dataset for preference alignment with response evolution</p>
    <p>🦁 Generate a dataset for preference alignment with response evolution and model pooling</p>
</div>

In [2]:
from huggingface_hub import login

login()

In [3]:
from huggingface_hub import create_repo

create_repo(repo_id="Tina-xxxx/huggingface-smol-course-preference-tuning-dataset", repo_type="dataset")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


RepoUrl('https://huggingface.co/datasets/Tina-xxxx/huggingface-smol-course-preference-tuning-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Tina-xxxx/huggingface-smol-course-preference-tuning-dataset')

## Install dependencies

Instead of transformers, you can also install `vllm` or `hf-inference-endpoints`.

In [ ]:
!pip install "distilabel[hf-transformers,outlines,instructor]"

## Start synthesizing

As we've seen in the previous notebook, we can create a distilabel pipeline for preference dataset generation. The bare minimum pipline is already provided. You can continue work on this pipeline to generate a large dataset for preference alignment. Swap out models, model providers and generation arguments to see how they affect the quality of the dataset. Experiment small, scale up later.

Check out the [distilabel components gallery](https://distilabel.argilla.io/latest/components-gallery/) for information about the processing classes and how to use them.

An example of loading data from the Hub instead of dictionaries is provided below.

```python
from datasets import load_dataset

with Pipeline(...) as pipeline:
    ...

if __name__ == "__main__:
    dataset = load_dataset("my-dataset", split="train")
    distiset = pipeline.run(dataset=dataset)
```

Don't forget to push your dataset to the Hub after running the pipeline!

In [5]:
from distilabel.llms import TransformersLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import GroupColumns, LoadDataFromDicts
from distilabel.steps.tasks import TextGeneration

with Pipeline() as pipeline:
    data = LoadDataFromDicts(data=[{"instruction": "What is synthetic data?"}])
    llm_a = TransformersLLM(model="HuggingFaceTB/SmolLM2-1.7B-Instruct")
    gen_a = TextGeneration(llm=llm_a)
    llm_b = TransformersLLM(model="Qwen/Qwen2.5-1.5B-Instruct")
    gen_b = TextGeneration(llm=llm_b)
    group = GroupColumns(columns=["generation"])
    data >> [gen_a, gen_b] >> group

if __name__ == "__main__":
    distiset = pipeline.run()
    distiset.push_to_hub("Tina-xxxx/huggingface-smol-course-preference-tuning-dataset")

[05/03/25 04:00:31] INFO     ['distilabel.pipeline'] 💾 Loading `_BatchManager` from cache:             ]8;id=180270;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=71326;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#967\967]8;;\
                             '/root/.cache/distilabel/pipelines/pipeline_load_data_from_dicts_0_text_ge            
                             neration_0_text_generation_1_group_columns_0/3e803650c76fd2bf3ca599624bec1            
                             6f2135a7401/executions/74a0299eb48ad0258727d61095bc0f45c7deacc8/batch_mana            
                             ger.json'                                                                             

                    INFO     ['distilabel.pipeline'] 💾 Loaded batch manager from cache doesn't contain ]8;id=684841;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py\base.py]8;;\:]8;id=247661;file:///usr/local/lib/python3.11/dist-packages/distilabel/pipeline/base.py#394\394]8;;\
                             any remaining data. Returning `Distiset` from cache data...                           

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/handlers.py", line 1492, in emit
    self.enqueue(self.prepare(record))
  File "/usr/lib/python3.11/logging/handlers.py", line 1450, in enqueue
    self.queue.put_nowait(record)
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 138, in put_nowait
    return self.put(obj, False)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 88, in put
    raise ValueError(f"Queue {self!r} is closed")
ValueError: Queue <multiprocessing.queues.Queue object at 0x7cb90022c910> is closed
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/

### Generate a dataset for preference alignment with response evolution

In [6]:
from distilabel.llms import TransformersLLM
from distilabel.steps.tasks import EvolQuality

llm = TransformersLLM(model="HuggingFaceTB/SmolLM2-1.7B-Instruct")
evol_quality = EvolQuality(llm=llm, num_evolutions=1)
evol_quality.load()

instruction = "What is synthetic data?"
completion = "Synthetic data is artificially generated data that mimics real-world usage."

result = next(evol_quality.process([{
    "instruction": instruction,
    "response": completion
}]))

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/handlers.py", line 1492, in emit
    self.enqueue(self.prepare(record))
  File "/usr/lib/python3.11/logging/handlers.py", line 1450, in enqueue
    self.queue.put_nowait(record)
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 138, in put_nowait
    return self.put(obj, False)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 88, in put
    raise ValueError(f"Queue {self!r} is closed")
ValueError: Queue <multiprocessing.queues.Queue object at 0x7cb90022c910> is closed
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  

In [8]:
print(result[0]["response"])
print(result[0]["evolved_response"])

Synthetic data is artificially generated data that mimics real-world usage.
Synthetic data refers to artificially created data designed to mimic real-world scenarios or patterns. It's often used for training machine learning models when collecting actual data may be impractical or impossible due to privacy concerns, cost, or other constraints. Synthetic data helps improve model performance while maintaining high accuracy levels.


### Generate a dataset for preference alignment with response evolution and model pooling

In [9]:
from distilabel.llms import TransformersLLM
from distilabel.steps.tasks import UltraFeedback

llm = TransformersLLM(model="HuggingFaceTB/SmolLM2-1.7B-Instruct")
ultrafeedback = UltraFeedback(llm=llm)
ultrafeedback.load()

instruction = "What is synthetic data?"
completion_a = "Synthetic data is artificially generated data that mimics real-world usage."
completion_b = "Synthetic data refers to data that has been generated artificially."

result = next(ultrafeedback.process([{
    "instruction": instruction,
    "generations": [completion_a, completion_b]
}]))

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/handlers.py", line 1492, in emit
    self.enqueue(self.prepare(record))
  File "/usr/lib/python3.11/logging/handlers.py", line 1450, in enqueue
    self.queue.put_nowait(record)
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 138, in put_nowait
    return self.put(obj, False)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 88, in put
    raise ValueError(f"Queue {self!r} is closed")
ValueError: Queue <multiprocessing.queues.Queue object at 0x7cb90022c910> is closed
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  

In [13]:
print(result[0]["ratings"])
print(result[0]["rationales"][0])
print(result[0]["rationales"][1])

[4, 3]
The first sentence accurately defines synthetic data as "artificially generated data" while the second sentence provides additional context about mimicking real-world usage. However, the second sentence could be more concise.
The first sentence correctly identifies synthetic data as artificial data, but the second sentence lacks clarity regarding what kind of artificial generation occurs. It would be beneficial to include specific examples or explanations.


## 🌯 That's a wrap

You've now seen how to generate a dataset for preference alignment. You could use this to:

- Generate a dataset for preference alignment.
- Create evaluation datasets for preference alignment.

Next

🏋️‍♂️ Fine-tune a model with preference alignment with a synthetic dataset based on the [preference tuning chapter](../../2_preference_alignment/README.md)
